# Design CTP-10 Aire DNA-MERFISH library for all super enhancers

by Pu Zheng

2021.4.5

Super-enhancers are called by ...

# Table of contents


> 0. [Minimum required packages and settings](#0)
>>
>> 0.1: [import required packages](#0.1)
>
> 1. [Extract region sequences](#1)

<a id='0.1'></a>
## 0.1 load required packages

In [1]:
%run "..\..\Startup_py3.py"
sys.path.append(r"..\..\..\..\Documents")

import ImageAnalysis3 as ia
%matplotlib notebook

from ImageAnalysis3 import *
print(os.getpid())

# library design specific tools
from ImageAnalysis3.library_tools import LibraryDesigner as ld
from ImageAnalysis3.library_tools import LibraryTools as lt
# biopython imports
from Bio import SeqIO
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord
from Bio.Blast.Applications import NcbiblastnCommandline
from Bio.Blast import NCBIXML

21036


<a id='1'></a>
# 1 Extract region sequences

In [2]:
## Some folders
# human genome
reference_folder = r'\\10.245.74.212\Chromatin_NAS_2\Libraries\Genomes\mouse\GRCm38_ensembl'
genome_folder = os.path.join(reference_folder, 'Genome')
# Library directories
pool_folder = r'\\10.245.74.212\Chromatin_NAS_2\Libraries\CTP-10_Aire'

In [3]:
resolution = 0
flanking = 10000
# folder for sub-pool
library_folder = os.path.join(pool_folder, f'Genes_TSS_DNA')
if not os.path.exists(library_folder):
    print(f"create library folder: {library_folder}")
    os.makedirs(library_folder)
# folder for fasta sequences
sequence_folder = os.path.join(library_folder, 'sequences')
if not os.path.exists(sequence_folder):
    print(f"create sequence folder: {sequence_folder}")
    os.makedirs(sequence_folder)
# folder to save result probes
report_folder = os.path.join(library_folder, 'reports')
if not os.path.exists(report_folder):
    print(f"create report folder: {report_folder}")
    os.makedirs(report_folder)
    
print(f"-- library_folder: {library_folder}")
print(f"-- sequence_folder: {sequence_folder}")
print(f"-- report_folder: {report_folder}")

-- library_folder: \\10.245.74.212\Chromatin_NAS_2\Libraries\CTP-10_Aire\Genes_TSS_DNA
-- sequence_folder: \\10.245.74.212\Chromatin_NAS_2\Libraries\CTP-10_Aire\Genes_TSS_DNA\sequences
-- report_folder: \\10.245.74.212\Chromatin_NAS_2\Libraries\CTP-10_Aire\Genes_TSS_DNA\reports


## 1.1 load gene list

In [4]:
gene_list_folder = os.path.join(pool_folder, 'Gene_list')
gene_list_filename = os.path.join(gene_list_folder, 'uniqued_clustered_genes_for_yuan_2021-04-22.txt')

In [11]:
import pandas as pd
gene_df = pd.read_csv(gene_list_filename, delimiter = "\t", header=None)
gene_df.columns = ['Cluster', 'Gene']

In [12]:
np.where(gene_df)

,Cluster,Gene
0,Immature MEC,Ccl21a
1,Immature MEC,Krt14
2,Immature MEC,Krt5
3,Immature MEC,Col6a1
4,Immature MEC,Lifr
...,...,...
204,Aire-stage,Ltf
205,Aire-stage,Clps
206,Aire-stage,Col1a1
207,Aire-stage,Gpx6


In [53]:
# load gene reference
reload(library_tools.references)
reload(library_tools.sequences)
ref_filename = os.path.join(reference_folder, 'Transcriptome', 'Mus_musculus.GRCm38.102.chr.gff3')
with library_tools.references.gff3_reader(ref_filename, auto_read=False, load_savefile=True) as ref_rd:
    ## example commands
    #infos = _handle.load_all()
    #ref_rd._load_headers()
    #gene_infos = ref_rd.load_gene_by_id('ENSMUSG00000064842')
    #gene_dict = ref_rd.parse_gene_info(gene_infos)
    #gene_info_dict = ref_rd._batch_parse_gene_info()
    ref_save_dict = ref_rd._save_to_file(overwrite=False)
# searching example
matched_gene = ref_rd._search_gene_by_id('gene:ENSMUSG00000089613')
print(matched_gene[0]['infos']['Name'])
matched_gene = ref_rd._search_gene_by_name('Ccl21a')
print(matched_gene[0]['infos']['ID'])

- loading from save_file: \\10.245.74.212\Chromatin_NAS_2\Libraries\Genomes\mouse\GRCm38_ensembl\Transcriptome\Mus_musculus.GRCm38.102.chr.pkl
opening ref_file: \\10.245.74.212\Chromatin_NAS_2\Libraries\Genomes\mouse\GRCm38_ensembl\Transcriptome\Mus_musculus.GRCm38.102.chr.gff3


In [60]:
from tqdm.notebook import tqdm

In [64]:
gene_dicts = []
for _gene in tqdm(gene_df['Gene']):
    if _gene == 'Lect1': # there is one gene that actually has different name
        _gene = "Cnmd" 
    _gds = ref_rd._search_gene_by_name(_gene)
    if len(_gds) == 1:
        gene_dicts.append(_gds[0])
    else:
        print(_gene)

  0%|          | 0/209 [00:00<?, ?it/s]

In [111]:
# convert reg_dict into reg_start_dict
reg_dicts = [library_tools.sequences.gene_dict_2_reg_dict(_gd) for _gd in gene_dicts]
tss_dicts = []
reg_size = 10000
for _rd in reg_dicts:
    _tss_d = {_k:_v for _k,_v in _rd.items()}
    if _tss_d['Strand'] == '+':
        _tss_d["Start"] = int(_rd['Start']) - int(reg_size/2)
        _tss_d["End"] = int(_rd['Start']) + int(reg_size/2)
        _tss_d['Region'] = f"{_rd['Chr']}:{_tss_d['Start']}-{_tss_d['End']}"
    else:
        _tss_d["Start"] = int(_rd['End']) - int(reg_size/2)
        _tss_d["End"] = int(_rd['End']) + int(reg_size/2)
        _tss_d['Region'] = f"{_rd['Chr']}:{_tss_d['Start']}-{_tss_d['End']}"
    # append
    tss_dicts.append(_tss_d)

In [112]:
reg_dicts[0], tss_dicts[0]

({'Chr': '4',
  'Start': '42772860',
  'End': '42773993',
  'Name': 'gene:ENSMUSG00000094686-Ccl21a',
  'Gene': 'Ccl21a',
  'Region': '4:42772860-42773993',
  'Strand': '-'},
 {'Chr': '4',
  'Start': 42768993,
  'End': 42778993,
  'Name': 'gene:ENSMUSG00000094686-Ccl21a',
  'Gene': 'Ccl21a',
  'Region': '4:42768993-42778993',
  'Strand': '-'})

In [86]:
reload(library_tools.references)
reload(library_tools.sequences)
seq_rd = library_tools.sequences.RNA_sequence_reader(genome_folder, flanking=flanking)
seq_rd.load_sequences()

-- load sequence: 1, size=195471971
-- load sequence: 10, size=130694993
-- load sequence: 11, size=122082543
-- load sequence: 12, size=120129022
-- load sequence: 13, size=120421639
-- load sequence: 14, size=124902244
-- load sequence: 15, size=104043685
-- load sequence: 16, size=98207768
-- load sequence: 17, size=94987271
-- load sequence: 18, size=90702639
-- load sequence: 19, size=61431566
-- load sequence: 2, size=182113224
-- load sequence: 3, size=160039680
-- load sequence: 4, size=156508116
-- load sequence: 5, size=151834684
-- load sequence: 6, size=149736546
-- load sequence: 7, size=145441459
-- load sequence: 8, size=129401213
-- load sequence: 9, size=124595110
-- load sequence: MT, size=16299
-- load sequence: X, size=171031299
-- load sequence: Y, size=91744698
-- load sequence: JH584299.1, size=953012
-- load sequence: GL456233.1, size=336933
-- load sequence: JH584301.1, size=259875
-- load sequence: GL456211.1, size=241735
-- load sequence: GL456350.1, size=227

In [113]:
for _tss_d in tss_dicts:
    seq_rd.find_sequence_for_region(_tss_d)
print(len(seq_rd.seq_dict))

-- searching among 1 references
-- a match found in record: 4.
-- searching among 1 references
-- a match found in record: 11.
-- searching among 1 references
-- a match found in record: 15.
-- searching among 1 references
-- a match found in record: 10.
-- searching among 1 references
-- a match found in record: 15.
-- searching among 1 references
-- a match found in record: 2.
-- searching among 1 references
-- a match found in record: 8.
-- searching among 1 references
-- a match found in record: 10.
-- searching among 1 references
-- a match found in record: 6.
-- searching among 1 references
-- a match found in record: 11.
-- searching among 1 references
-- a match found in record: 17.
-- searching among 1 references
-- a match found in record: 16.
-- searching among 1 references
-- a match found in record: 3.
-- searching among 1 references
-- a match found in record: 7.
-- searching among 1 references
-- a match found in record: 17.
-- searching among 1 references
-- a match fou

In [116]:
seq_rd.save_sequences(sequence_folder)

-- saving sequences into folder: \\10.245.74.212\Chromatin_NAS_2\Libraries\CTP-10_Aire\Genes_TSS_DNA\sequences
-- save to file: \\10.245.74.212\Chromatin_NAS_2\Libraries\CTP-10_Aire\Genes_TSS_DNA\sequences\Ccl21a_reg_1.fasta
-- save to file: \\10.245.74.212\Chromatin_NAS_2\Libraries\CTP-10_Aire\Genes_TSS_DNA\sequences\Krt14_reg_1.fasta
-- save to file: \\10.245.74.212\Chromatin_NAS_2\Libraries\CTP-10_Aire\Genes_TSS_DNA\sequences\Krt5_reg_1.fasta
-- save to file: \\10.245.74.212\Chromatin_NAS_2\Libraries\CTP-10_Aire\Genes_TSS_DNA\sequences\Col6a1_reg_1.fasta
-- save to file: \\10.245.74.212\Chromatin_NAS_2\Libraries\CTP-10_Aire\Genes_TSS_DNA\sequences\Lifr_reg_1.fasta
-- save to file: \\10.245.74.212\Chromatin_NAS_2\Libraries\CTP-10_Aire\Genes_TSS_DNA\sequences\Itga6_reg_1.fasta
-- save to file: \\10.245.74.212\Chromatin_NAS_2\Libraries\CTP-10_Aire\Genes_TSS_DNA\sequences\Col4a1_reg_1.fasta
-- save to file: \\10.245.74.212\Chromatin_NAS_2\Libraries\CTP-10_Aire\Genes_TSS_DNA\sequences\As

-- save to file: \\10.245.74.212\Chromatin_NAS_2\Libraries\CTP-10_Aire\Genes_TSS_DNA\sequences\Ly6c2_reg_1.fasta
-- save to file: \\10.245.74.212\Chromatin_NAS_2\Libraries\CTP-10_Aire\Genes_TSS_DNA\sequences\Cxcl13_reg_1.fasta
-- save to file: \\10.245.74.212\Chromatin_NAS_2\Libraries\CTP-10_Aire\Genes_TSS_DNA\sequences\Penk_reg_1.fasta
-- save to file: \\10.245.74.212\Chromatin_NAS_2\Libraries\CTP-10_Aire\Genes_TSS_DNA\sequences\Ecm1_reg_1.fasta
-- save to file: \\10.245.74.212\Chromatin_NAS_2\Libraries\CTP-10_Aire\Genes_TSS_DNA\sequences\Sprr1b_reg_1.fasta
-- save to file: \\10.245.74.212\Chromatin_NAS_2\Libraries\CTP-10_Aire\Genes_TSS_DNA\sequences\Ly6c1_reg_1.fasta
-- save to file: \\10.245.74.212\Chromatin_NAS_2\Libraries\CTP-10_Aire\Genes_TSS_DNA\sequences\Hbb-y_reg_1.fasta
-- save to file: \\10.245.74.212\Chromatin_NAS_2\Libraries\CTP-10_Aire\Genes_TSS_DNA\sequences\Ccl1_reg_1.fasta
-- save to file: \\10.245.74.212\Chromatin_NAS_2\Libraries\CTP-10_Aire\Genes_TSS_DNA\sequences\Kl

<a id='2'></a>
# 2. Design probe targeting sequences by probe_designer

run probe_designer, remeber to clear the memory usage because each table should take ~32GB

<a id='2.1'></a>
## 2.1 Construct count table with all the 17-mers in the genome

Only do this if you don't have pre-built 17-mer

However you can do almost the same thing for your own library during quality check

This library requires mm10 genome

In [117]:
overwrite_table = False

### construct map for whole genome

In [118]:
reload(library_tools.design)
 
genome_table_file = os.path.join(reference_folder, 'GRCm38_genome_17w.npy')

if not os.path.exists(genome_table_file) or overwrite_table:
    # genome
    _genome_filenames = [os.path.join(genome_folder, _fl) 
         for _fl in os.listdir(genome_folder) 
         if _fl.split(os.extsep)[-1]=='fasta' or _fl.split(os.extsep)[-1]=='fa']
    print(len(_genome_filenames))

    ct = library_tools.design.countTable(word=17,save_file=genome_table_file, 
                       sparse=False)
    ct.verbose=True

    ct.read(_genome_filenames) # read sequences from fasta files

    ct.consume_loaded(num_threads=24) # convert sequences into integers

    ct.complete(verbose=True)

    ct.save()

    # clear RAM if contructed countable 
    del(ct)

### construct map for transcriptome

In [119]:
from tqdm import tqdm
# transcriptome
transcriptome_folder = os.path.join(reference_folder, 'Transcriptome')

transcriptome_table_file = os.path.join(reference_folder, 'GRCm38_transcriptome_17w.npy')

if not os.path.exists(transcriptome_table_file) or overwrite_table:
    # transcriptome
    _transcriptome_filenames = [os.path.join(transcriptome_folder, _fl) 
         for _fl in os.listdir(transcriptome_folder) 
         if _fl.split(os.extsep)[-1]=='fasta' or _fl.split(os.extsep)[-1]=='fa']
    print(len(_transcriptome_filenames))
    
    ct = library_tools.design.countTable(word=17,save_file=transcriptome_table_file, 
                       sparse=False)
    ct.verbose=True

    ct.read(_transcriptome_filenames) # read sequences from fasta files

    ct.consume_loaded(num_threads=24) # convert sequences into integers

    ct.complete(verbose=True)

    ct.save()
    
    # clear RAM if contructed countable 
    del(ct)

### construct map for repeats from RepBase

In [120]:
from tqdm import tqdm
# repeat
repeat_folder = r'\\10.245.74.212\Chromatin_NAS_2\Libraries\Genomes\Repbase'

repeat_table_file = os.path.join(reference_folder, 'Repbase_v2603_repeat_17w.npy')

if not os.path.exists(repeat_table_file) or overwrite_table:
    # repeat
    _repeat_filenames = [os.path.join(repeat_folder, _fl) 
         for _fl in os.listdir(repeat_folder) 
         if _fl.split(os.extsep)[-1]=='fasta' or _fl.split(os.extsep)[-1]=='fa']
    print(len(_repeat_filenames))
    
    ct = library_tools.design.countTable(word=17,save_file=repeat_table_file, 
                       sparse=False)
    ct.verbose=True

    ct.read(_repeat_filenames) # read sequences from fasta files

    ct.consume_loaded(num_threads=24) # convert sequences into integers

    ct.complete(verbose=True)

    ct.save()
    
    # clear RAM if contructed countable 
    del(ct)

<a id='2.2'></a>
## 2.2 Design probes

In [121]:
# required parameters
resolution = 0

## required folders
## Some folders
# human genome
reference_folder = r'\\10.245.74.212\Chromatin_NAS_2\Libraries\Genomes\mouse\GRCm38_ensembl'
genome_folder = os.path.join(reference_folder, 'Genome')
# Library directories
pool_folder = r'\\10.245.74.212\Chromatin_NAS_2\Libraries\CTP-10_Aire'

library_folder = os.path.join(pool_folder, f'Genes_TSS_DNA')
sequence_folder = os.path.join(library_folder, 'sequences')
report_folder = os.path.join(library_folder, 'reports')

In [122]:
# requires pre_defined genome_folder and library_folder
# Indices
genome_index = os.path.join(reference_folder, 'GRCm38_genome_17w.npy')
transcriptome_index = os.path.join(reference_folder, 'GRCm38_transcriptome_17w.npy') 
repeat_index = os.path.join(reference_folder, 'Repbase_v2603_repeat_17w.npy')
#ref_merfish_index = os.path.join(reference_folder, 'M1_meng_MERFISH_17w.npy') # merfish designed by Meng
# get input files 
input_files = glob.glob(os.path.join(sequence_folder, '*.fasta'))

print(f"{len(input_files)} regions loaded to design probes.")

if not os.path.exists(report_folder):
    os.makedirs(report_folder)
    
# filename to save probe reports
save_file = os.path.join(report_folder, f'merged_probes.pbr')
print(save_file)

209 regions loaded to design probes.
\\10.245.74.212\Chromatin_NAS_2\Libraries\CTP-10_Aire\Genes_TSS_DNA\reports\merged_probes.pbr


### create pb_designer class

In [123]:
reload(library_tools)
reload(library_tools.design)

pb_designer = library_tools.design.pb_reports_class(
    sequence_dic={'file':input_files,
                  'rev_com':True, # design two strands
                  'two_stranded':True},
    map_dic={'genome':{'file':genome_index,'rev_com':False,'two_stranded':True},
             'transcriptome':{'file':transcriptome_index,'rev_com':True,'two_stranded':False},
             'rep_genome':{'file':repeat_index,'rev_com':False,'two_stranded':True},
             'self_sequences':{'file':input_files,'force_list':True,'rev_com':False,'two_stranded':True},
             #'ref_merfish':{'file':ref_merfish_index,'rev_com':False,'two_stranded':True},
             },
        save_file=save_file,
    params_dic={'word_size':17,'pb_len':40,'buffer_len':1,'max_count':2**16-1,
                'check_on_go': False, # whether automatically check probes
                'auto': False, # whether automatically convert reference maps
               },
    check_dic={('genome','self_sequences'): 25,
               'rep_genome': 0,
               'transcriptome': 14,
               #'ref_merfish': 14,
               'gc':[0.25,0.85],'tm': 70,
               }
    )
print(pb_designer)
pb_designer.load_from_file(load_probes_only=True)


Probe designer derived from Bogdan Bintu:
https://github.com/BogdanBintu/ChromatinImaging/blob/master/LibraryDesign/LibraryDesigner.py
by Pu Zheng, 2020.11

Major changes:
    1. allow design of two strands
    2. separate reverse_complement (rev_com) and from two strands (two_stranded) as 
    two different inputs for map_dic and sequence_dic
    3. replace 'local_genome' with 'self_sequences' to be more explicit, and only 
    exclude the counts for the corresponding self_sequence within each input. 

Key information:
    - number of input_sequence(s): 209
    - save_file location: \\10.245.74.212\Chromatin_NAS_2\Libraries\CTP-10_Aire\Genes_TSS_DNA\reports\merged_probes.pbr

- Fail to load from savefile: \\10.245.74.212\Chromatin_NAS_2\Libraries\CTP-10_Aire\Genes_TSS_DNA\reports\merged_probes.pbr, file doesn't exist.


False

### calculate probe reports

In [124]:
reload(library_tools)
reload(library_tools.design)
pb_designer.computeOTmaps() # load the tables 
pb_designer.compute_pb_report() # design candidate probes

-- setting attribute: map_genome
--- finish map_genome in 89.868s.
-- setting attribute: map_transcriptome
--- finish map_transcriptome in 93.490s.
-- setting attribute: map_rep_genome
--- finish map_rep_genome in 88.290s.
Time(s): 271.6564977169037
- Designing targeting sequence for 209 regions
-- designing region: 9:104322748-104352748_reg_Acpp:- 0, input file: \\10.245.74.212\Chromatin_NAS_2\Libraries\CTP-10_Aire\Genes_TSS_DNA\sequences\Acpp_reg_1.fasta
-- setting attribute: map_self_sequences
- Mapping no. of seqs: 1
--- finish map_self_sequences in 0.118s.
in 15.720s.
-- designing region: 8:124554706-124584706_reg_Agt:- 1, input file: \\10.245.74.212\Chromatin_NAS_2\Libraries\CTP-10_Aire\Genes_TSS_DNA\sequences\Agt_reg_1.fasta
-- setting attribute: map_self_sequences
- Mapping no. of seqs: 1
--- finish map_self_sequences in 0.150s.
in 16.387s.
-- designing region: 11:61192537-61222537_reg_Aldh3a1:+ 2, input file: \\10.245.74.212\Chromatin_NAS_2\Libraries\CTP-10_Aire\Genes_TSS_DNA\

in 16.367s.
-- designing region: 11:82020571-82050571_reg_Ccl2:+ 28, input file: \\10.245.74.212\Chromatin_NAS_2\Libraries\CTP-10_Aire\Genes_TSS_DNA\sequences\Ccl2_reg_1.fasta
-- setting attribute: map_self_sequences
- Mapping no. of seqs: 1
--- finish map_self_sequences in 0.118s.
in 16.342s.
-- designing region: 11:83578087-83608087_reg_Ccl6:- 29, input file: \\10.245.74.212\Chromatin_NAS_2\Libraries\CTP-10_Aire\Genes_TSS_DNA\sequences\Ccl6_reg_1.fasta
-- setting attribute: map_self_sequences
- Mapping no. of seqs: 1
--- finish map_self_sequences in 0.126s.
in 16.296s.
-- designing region: 11:83563636-83593636_reg_Ccl9:- 30, input file: \\10.245.74.212\Chromatin_NAS_2\Libraries\CTP-10_Aire\Genes_TSS_DNA\sequences\Ccl9_reg_1.fasta
-- setting attribute: map_self_sequences
- Mapping no. of seqs: 1
--- finish map_self_sequences in 0.137s.
in 16.400s.
-- designing region: 18:36711798-36741798_reg_Cd14:- 31, input file: \\10.245.74.212\Chromatin_NAS_2\Libraries\CTP-10_Aire\Genes_TSS_DNA\se

in 16.232s.
-- designing region: 9:83791277-83821277_reg_Elovl4:- 57, input file: \\10.245.74.212\Chromatin_NAS_2\Libraries\CTP-10_Aire\Genes_TSS_DNA\sequences\Elovl4_reg_1.fasta
-- setting attribute: map_self_sequences
- Mapping no. of seqs: 1
--- finish map_self_sequences in 0.134s.
in 16.259s.
-- designing region: 6:71184827-71214827_reg_Fabp1:+ 58, input file: \\10.245.74.212\Chromatin_NAS_2\Libraries\CTP-10_Aire\Genes_TSS_DNA\sequences\Fabp1_reg_1.fasta
-- setting attribute: map_self_sequences
- Mapping no. of seqs: 1
--- finish map_self_sequences in 0.150s.
in 16.358s.
-- designing region: 11:43586540-43616540_reg_Fabp6:- 59, input file: \\10.245.74.212\Chromatin_NAS_2\Libraries\CTP-10_Aire\Genes_TSS_DNA\sequences\Fabp6_reg_1.fasta
-- setting attribute: map_self_sequences
- Mapping no. of seqs: 1
--- finish map_self_sequences in 0.137s.
in 16.270s.
-- designing region: 11:58786960-58816960_reg_Fam183b:- 60, input file: \\10.245.74.212\Chromatin_NAS_2\Libraries\CTP-10_Aire\Genes_T

in 16.247s.
-- designing region: 7:142563143-142593143_reg_H19:- 86, input file: \\10.245.74.212\Chromatin_NAS_2\Libraries\CTP-10_Aire\Genes_TSS_DNA\sequences\H19_reg_1.fasta
-- setting attribute: map_self_sequences
- Mapping no. of seqs: 1
--- finish map_self_sequences in 0.133s.
in 16.397s.
-- designing region: 7:30909681-30939681_reg_Hamp2:- 87, input file: \\10.245.74.212\Chromatin_NAS_2\Libraries\CTP-10_Aire\Genes_TSS_DNA\sequences\Hamp2_reg_1.fasta
-- setting attribute: map_self_sequences
- Mapping no. of seqs: 1
--- finish map_self_sequences in 0.135s.
in 16.415s.
-- designing region: 11:32281489-32311489_reg_Hba-a2:+ 88, input file: \\10.245.74.212\Chromatin_NAS_2\Libraries\CTP-10_Aire\Genes_TSS_DNA\sequences\Hba-a2_reg_1.fasta
-- setting attribute: map_self_sequences
- Mapping no. of seqs: 1
--- finish map_self_sequences in 0.127s.
in 16.322s.
-- designing region: 7:103838216-103868216_reg_Hbb-y:- 89, input file: \\10.245.74.212\Chromatin_NAS_2\Libraries\CTP-10_Aire\Genes_TSS_

in 16.427s.
-- designing region: 15:101877920-101907920_reg_Krt76:- 115, input file: \\10.245.74.212\Chromatin_NAS_2\Libraries\CTP-10_Aire\Genes_TSS_DNA\sequences\Krt76_reg_1.fasta
-- setting attribute: map_self_sequences
- Mapping no. of seqs: 1
--- finish map_self_sequences in 0.118s.
in 16.401s.
-- designing region: 15:101854705-101884705_reg_Krt77:- 116, input file: \\10.245.74.212\Chromatin_NAS_2\Libraries\CTP-10_Aire\Genes_TSS_DNA\sequences\Krt77_reg_1.fasta
-- setting attribute: map_self_sequences
- Mapping no. of seqs: 1
--- finish map_self_sequences in 0.134s.
in 16.517s.
-- designing region: 11:100178246-100208246_reg_Krt9:- 117, input file: \\10.245.74.212\Chromatin_NAS_2\Libraries\CTP-10_Aire\Genes_TSS_DNA\sequences\Krt9_reg_1.fasta
-- setting attribute: map_self_sequences
- Mapping no. of seqs: 1
--- finish map_self_sequences in 0.131s.
in 16.596s.
-- designing region: 7:30772896-30802896_reg_Krtdap:+ 118, input file: \\10.245.74.212\Chromatin_NAS_2\Libraries\CTP-10_Aire\G

in 16.270s.
-- designing region: 4:147985722-148015722_reg_Nppa:+ 144, input file: \\10.245.74.212\Chromatin_NAS_2\Libraries\CTP-10_Aire\Genes_TSS_DNA\sequences\Nppa_reg_1.fasta
-- setting attribute: map_self_sequences
- Mapping no. of seqs: 1
--- finish map_self_sequences in 0.126s.
in 16.270s.
-- designing region: 6:49807710-49837710_reg_Npy:+ 145, input file: \\10.245.74.212\Chromatin_NAS_2\Libraries\CTP-10_Aire\Genes_TSS_DNA\sequences\Npy_reg_1.fasta
-- setting attribute: map_self_sequences
- Mapping no. of seqs: 1
--- finish map_self_sequences in 0.131s.
in 16.269s.
-- designing region: 9:37537904-37567904_reg_Nrgn:- 146, input file: \\10.245.74.212\Chromatin_NAS_2\Libraries\CTP-10_Aire\Genes_TSS_DNA\sequences\Nrgn_reg_1.fasta
-- setting attribute: map_self_sequences
- Mapping no. of seqs: 1
--- finish map_self_sequences in 0.134s.
in 16.259s.
-- designing region: 10:102475486-102505486_reg_Nts:- 147, input file: \\10.245.74.212\Chromatin_NAS_2\Libraries\CTP-10_Aire\Genes_TSS_DNA\

in 16.289s.
-- designing region: 14:51131785-51161785_reg_Rnase1:- 173, input file: \\10.245.74.212\Chromatin_NAS_2\Libraries\CTP-10_Aire\Genes_TSS_DNA\sequences\Rnase1_reg_1.fasta
-- setting attribute: map_self_sequences
- Mapping no. of seqs: 1
--- finish map_self_sequences in 0.134s.
in 16.355s.
-- designing region: 14:51076077-51106077_reg_Rnase4:+ 174, input file: \\10.245.74.212\Chromatin_NAS_2\Libraries\CTP-10_Aire\Genes_TSS_DNA\sequences\Rnase4_reg_1.fasta
-- setting attribute: map_self_sequences
- Mapping no. of seqs: 1
--- finish map_self_sequences in 0.131s.
in 16.271s.
-- designing region: 12:26441452-26471452_reg_Rsad2:- 175, input file: \\10.245.74.212\Chromatin_NAS_2\Libraries\CTP-10_Aire\Genes_TSS_DNA\sequences\Rsad2_reg_1.fasta
-- setting attribute: map_self_sequences
- Mapping no. of seqs: 1
--- finish map_self_sequences in 0.134s.
in 16.288s.
-- designing region: 17:31262383-31292383_reg_Rsph1:- 176, input file: \\10.245.74.212\Chromatin_NAS_2\Libraries\CTP-10_Aire\G

in 16.230s.
-- designing region: 16:25668763-25698763_reg_Trp63:+ 202, input file: \\10.245.74.212\Chromatin_NAS_2\Libraries\CTP-10_Aire\Genes_TSS_DNA\sequences\Trp63_reg_1.fasta
-- setting attribute: map_self_sequences
- Mapping no. of seqs: 1
--- finish map_self_sequences in 0.134s.
in 16.237s.
-- designing region: 7:143079642-143109642_reg_Trpm5:- 203, input file: \\10.245.74.212\Chromatin_NAS_2\Libraries\CTP-10_Aire\Genes_TSS_DNA\sequences\Trpm5_reg_1.fasta
-- setting attribute: map_self_sequences
- Mapping no. of seqs: 1
--- finish map_self_sequences in 0.134s.
in 16.274s.
-- designing region: 4:116152601-116182601_reg_Tspan1:- 204, input file: \\10.245.74.212\Chromatin_NAS_2\Libraries\CTP-10_Aire\Genes_TSS_DNA\sequences\Tspan1_reg_1.fasta
-- setting attribute: map_self_sequences
- Mapping no. of seqs: 1
--- finish map_self_sequences in 0.136s.
in 16.270s.
-- designing region: 9:44788072-44818072_reg_Ttc36:- 205, input file: \\10.245.74.212\Chromatin_NAS_2\Libraries\CTP-10_Aire\Ge

### check probes

In [125]:
pbs, pb_scores = pb_designer.check_probes()

-- check region:0 9:104322748-104352748_reg_Acpp:-, 58448 candidate probes
--- 30346 probes passed check_dic, GC and Tm selection.
finish in 2.513s, 422 probes kept.
-- check region:1 8:124554706-124584706_reg_Agt:-, 53018 candidate probes
--- 25225 probes passed check_dic, GC and Tm selection.
finish in 2.080s, 352 probes kept.
-- check region:2 11:61192537-61222537_reg_Aldh3a1:+, 59118 candidate probes
--- 31353 probes passed check_dic, GC and Tm selection.
finish in 2.407s, 432 probes kept.
-- check region:3 4:49534546-49564546_reg_Aldob:-, 57512 candidate probes
--- 28730 probes passed check_dic, GC and Tm selection.
finish in 2.335s, 413 probes kept.
-- check region:4 5:149249767-149279767_reg_Alox5ap:+, 59050 candidate probes
--- 27984 probes passed check_dic, GC and Tm selection.
finish in 2.346s, 397 probes kept.
-- check region:5 1:87086606-87116606_reg_Alpi:-, 59514 candidate probes
--- 24857 probes passed check_dic, GC and Tm selection.
finish in 2.151s, 396 probes kept.
-- 

--- 25355 probes passed check_dic, GC and Tm selection.
finish in 2.105s, 400 probes kept.
-- check region:50 7:27104909-27134909_reg_Cyp2f2:+, 58014 candidate probes
--- 21187 probes passed check_dic, GC and Tm selection.
finish in 1.990s, 348 probes kept.
-- check region:51 13:55473111-55503111_reg_Dbn1:-, 59344 candidate probes
--- 29821 probes passed check_dic, GC and Tm selection.
finish in 2.319s, 433 probes kept.
-- check region:52 3:55227364-55257364_reg_Dclk1:+, 59692 candidate probes
--- 29249 probes passed check_dic, GC and Tm selection.
finish in 2.369s, 441 probes kept.
-- check region:53 12:109437823-109467823_reg_Dlk1:+, 59602 candidate probes
--- 32565 probes passed check_dic, GC and Tm selection.
finish in 2.497s, 466 probes kept.
-- check region:54 6:122611410-122641410_reg_Dppa3:+, 56332 candidate probes
--- 17343 probes passed check_dic, GC and Tm selection.
finish in 1.831s, 276 probes kept.
-- check region:55 3:95724569-95754569_reg_Ecm1:-, 59310 candidate probes


finish in 2.389s, 476 probes kept.
-- check region:99 1:72859884-72889884_reg_Igfbp5:-, 59572 candidate probes
--- 31685 probes passed check_dic, GC and Tm selection.
finish in 2.407s, 454 probes kept.
-- check region:100 15:102129362-102159362_reg_Igfbp6:+, 58934 candidate probes
--- 25780 probes passed check_dic, GC and Tm selection.
finish in 2.200s, 387 probes kept.
-- check region:101 7:142728381-142758381_reg_Ins2:-, 56480 candidate probes
--- 22308 probes passed check_dic, GC and Tm selection.
finish in 1.993s, 327 probes kept.
-- check region:102 4:156185796-156215796_reg_Isg15:-, 58976 candidate probes
--- 28669 probes passed check_dic, GC and Tm selection.
finish in 2.325s, 420 probes kept.
-- check region:103 2:71730616-71760616_reg_Itga6:+, 59210 candidate probes
--- 32143 probes passed check_dic, GC and Tm selection.
finish in 2.411s, 459 probes kept.
-- check region:104 11:115499387-115529387_reg_Jpt1:-, 58398 candidate probes
--- 21333 probes passed check_dic, GC and Tm 

finish in 2.175s, 396 probes kept.
-- check region:148 5:120899536-120929536_reg_Oas1d:+, 58068 candidate probes
--- 13068 probes passed check_dic, GC and Tm selection.
finish in 1.818s, 261 probes kept.
-- check region:149 5:120846421-120876421_reg_Oas1h:+, 57490 candidate probes
--- 17356 probes passed check_dic, GC and Tm selection.
finish in 1.928s, 314 probes kept.
-- check region:150 15:89364587-89394587_reg_Odf3b:-, 59686 candidate probes
--- 27571 probes passed check_dic, GC and Tm selection.
finish in 2.262s, 450 probes kept.
-- check region:151 2:28177992-28207992_reg_Olfm1:+, 59524 candidate probes
--- 32848 probes passed check_dic, GC and Tm selection.
finish in 2.547s, 452 probes kept.
-- check region:152 16:96452606-96482606_reg_Pcp4:+, 59466 candidate probes
--- 31074 probes passed check_dic, GC and Tm selection.
finish in 2.366s, 450 probes kept.
-- check region:153 4:115073708-115103708_reg_Pdzk1ip1:+, 59480 candidate probes
--- 32449 probes passed check_dic, GC and Tm

finish in 2.161s, 380 probes kept.
-- check region:197 17:31129282-31159282_reg_Tff2:-, 28996 candidate probes
--- 14229 probes passed check_dic, GC and Tm selection.
finish in 1.284s, 196 probes kept.
-- check region:198 17:31114646-31144646_reg_Tff3:-, 59642 candidate probes
--- 28686 probes passed check_dic, GC and Tm selection.
finish in 2.276s, 431 probes kept.
-- check region:199 11:118340740-118370740_reg_Timp2:-, 59702 candidate probes
--- 30714 probes passed check_dic, GC and Tm selection.
finish in 2.427s, 433 probes kept.
-- check region:200 16:56871166-56901166_reg_Tmem45a:-, 59382 candidate probes
--- 20952 probes passed check_dic, GC and Tm selection.
finish in 1.963s, 325 probes kept.
-- check region:201 15:54263484-54293484_reg_Tnfrsf11b:-, 59462 candidate probes
--- 25487 probes passed check_dic, GC and Tm selection.
finish in 2.295s, 378 probes kept.
-- check region:202 16:25668763-25698763_reg_Trp63:+, 59652 candidate probes
--- 22892 probes passed check_dic, GC and 

### save probes

In [46]:
overwrite_savefile = True 
if not os.path.exists(pb_designer.save_file) or overwrite_savefile:
    #pb_designer.plots()
    #pb_designer.save_csv()
    pb_designer.save_to_file()
print(f"-- number of probes kept: {len(pb_designer.kept_probes)}")

- Save reports into file: \\10.245.74.212\Chromatin_NAS_2\Libraries\CTP-10_Aire\SuperEnhancers\reports\merged_probes.pbr
-- number of probes kept: 447331


'ATGTTACATGTATATGTGAGGTACGGTATGCACACATGCA'

In [126]:
from Bio.SeqUtils import MeltingTemp


In [133]:
MeltingTemp.Tm_NN(list(pb_designer.kept_probes.keys())[0].decode(), Na=330)

73.4869025515149

In [134]:
from Bio.SeqUtils import GC


In [136]:
?GC